# Scheduling Employees for Shifts

Problem Description (Scheduling Employees for Shifts)

You manage a company that operates 7 days a week with 3 shifts per day (morning, afternoon, night). You have 20 employees, each with different skills and availability. The goal is to assign employees to shifts to:

Meet the required number of employees per shift.


Ensure each shift has the minimum skill levels covered.


Limit the total working hours per employee.


Respect employee unavailability and preferences.


Minimize total overtime.



Variables & Data

    Employees: 20 people, each with skills (skill levels 1-5 in categories: cashier, stocker, cleaner).

    Shifts: 7 days * 3 shifts/day = 21 shifts.

    Required employees per shift and skill minimums.

    Employee availability: which shifts an employee can work.

    Maximum working hours per week (e.g., 40 hours).

    Shift length fixed at 8 hours.

In [4]:
import pulp
import random

# Seed for reproducibility
random.seed(123)

# Define sets
employees = [f'Emp_{i+1}' for i in range(20)]
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
shifts = ['Morning', 'Afternoon', 'Night']

# All shifts as tuples (day, shift)
all_shifts = [(d, s) for d in days for s in shifts]

# Parameters
shift_length = 8  # hours per shift
max_hours_per_week = 40

# Required number of employees per shift
required_employees = {
    (d, s): random.randint(3, 6) for d in days for s in shifts
}

# Skill categories
skills = ['cashier', 'stocker', 'cleaner']

# Minimum total skill level required per shift (sum across employees assigned)
min_skill_required = {
    (d, s): {
        'cashier': random.randint(4, 8),
        'stocker': random.randint(3, 7),
        'cleaner': random.randint(2, 5)
    }
    for d in days for s in shifts
}

# Employee skills (each skill 1 to 5)
employee_skills = {
    e: {
        'cashier': random.randint(1, 5),
        'stocker': random.randint(1, 5),
        'cleaner': random.randint(1, 5)
    }
    for e in employees
}

# Employee availability: True if employee can work the shift, False otherwise
employee_availability = {
    e: {
        (d, s): random.choice([True] * 8 + [False] * 3)  # 8/11 chance available
        for d in days for s in shifts
    }
    for e in employees
}

# Employee preferred shifts (soft constraints)
employee_preference = {
    e: {
        (d, s): random.choice([True] * 7 + [False] * 4)  # 7/11 chance preferred
        for d in days for s in shifts
    }
    for e in employees
}

# Create model
model = pulp.LpProblem("Employee_Shift_Scheduling", pulp.LpMinimize)

# Decision variables: x[e,d,s] = 1 if employee e works shift (d,s)
x = pulp.LpVariable.dicts(
    "shift",
    ((e, d, s) for e in employees for d in days for s in shifts),
    cat='Binary'
)

# Overtime variables: overtime_hours[e]
overtime_hours = pulp.LpVariable.dicts("overtime", employees, lowBound=0, cat='Continuous')

# Objective: minimize total overtime + penalty for assigning employees to non-preferred shifts
penalty_non_preferred = 10  # penalty cost per non-preferred shift assigned

model += (
    pulp.lpSum(overtime_hours[e] for e in employees) +
    penalty_non_preferred * pulp.lpSum(
        x[(e, d, s)] * (1 - int(employee_preference[e][(d, s)]))
        for e in employees for d in days for s in shifts
    ),
    "Minimize_Overtime_and_Preference_Violations"
)

# Constraints:

# 1. Staffing level: enough employees per shift
for d, s in all_shifts:
    model += (
        pulp.lpSum(x[(e, d, s)] for e in employees) >= required_employees[(d, s)],
        f"Staffing_{d}_{s}"
    )

# 2. Skill coverage per shift (sum of skills of assigned employees)
for d, s in all_shifts:
    for skill in skills:
        model += (
            pulp.lpSum(employee_skills[e][skill] * x[(e, d, s)] for e in employees) >= min_skill_required[(d, s)][skill],
            f"Skill_{skill}_{d}_{s}"
        )

# 3. Employee availability
for e in employees:
    for d, s in all_shifts:
        if not employee_availability[e][(d, s)]:
            model += x[(e, d, s)] == 0, f"Availability_{e}_{d}_{s}"

# 4. Max working hours per employee (shift_length * sum shifts) <= max_hours + overtime
for e in employees:
    model += (
        pulp.lpSum(shift_length * x[(e, d, s)] for d in days for s in shifts) <= max_hours_per_week + overtime_hours[e],
        f"Max_hours_{e}"
    )

# 5. No double shifts: employee can only work one shift per day
for e in employees:
    for d in days:
        model += (
            pulp.lpSum(x[(e, d, s)] for s in shifts) <= 1,
            f"One_shift_per_day_{e}_{d}"
        )

# 6. Rest time: No consecutive night to morning shifts (simplified constraint)
for e in employees:
    for i in range(len(days)-1):
        day = days[i]
        next_day = days[i+1]
        model += (
            x[(e, day, 'Night')] + x[(e, next_day, 'Morning')] <= 1,
            f"Rest_night_to_morning_{e}_{day}_{next_day}"
        )

# 7. Optional: Maximum number of night shifts per employee (e.g., max 3 per week)
max_night_shifts = 3
for e in employees:
    model += (
        pulp.lpSum(x[(e, d, 'Night')] for d in days) <= max_night_shifts,
        f"Max_night_shifts_{e}"
    )

# Solve the problem
print("Solving scheduling problem...")
model.solve()

print("Status:", pulp.LpStatus[model.status])

# Output schedule summary
for d in days:
    print(f"\nSchedule for {d}:")
    for s in shifts:
        assigned = [e for e in employees if pulp.value(x[(e, d, s)]) > 0.5]
        print(f"  {s}: {', '.join(assigned)}")

# Print overtime per employee
print("\nOvertime hours per employee:")
for e in employees:
    print(f"{e}: {pulp.value(overtime_hours[e]):.2f} hours")

Solving scheduling problem...
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/roniya/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/120470f65c6e400096fd40c4e93501d9-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/120470f65c6e400096fd40c4e93501d9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 490 COLUMNS
At line 4532 RHS
At line 5018 BOUNDS
At line 5439 ENDATA
Problem MODEL has 485 rows, 440 columns and 3021 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 20 - 0.00 seconds
Cgl0002I 101 variables fixed
Cgl0003I 0 fixed, 0 tightened bounds, 6 strengthened rows, 0 substitutions
Cgl0004I processed model has 315 rows, 339 columns (319 integer (319 of which binary)) and 2171 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 6.661